<a href="https://colab.research.google.com/github/codeREXus/langchain-learnings/blob/main/Langchain5_chromaDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q chromaDB

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 63.2 MB/s eta 0

In [ ]:
import chromadb
from chromadb.utils import embedding_functions

ef= embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name ="all-MiniLm-L6-V2"
)

In [8]:
client= chromadb.Client()
collection = client.create_collection(
    name='document',
    embedding_function=ef,
    metadata={"description": "Used to demo filtering in ChromaDB"}
)
print(f"created collections{collection}")

created collectionsCollection(name=document)


###adding docs

In [23]:
collection.add(
    documents=[
        "This is a document about LangChain",
        "This is a reading about LlamaIndex",
        "This is a book about Python",
        "This is a document about pandas",
        "This is another document about LangChain"
    ],
    metadatas=[
        {"source": "langchain.com", "version": 0.1},
        {"source": "llamaindex.ai", "version": 0.2},
        {"source": "python.org", "version": 0.3},
        {"source": "pandas.pydata.org", "version": 0.4},
        {"source": "langchain.com", "version": 0.5},
    ],
    ids=["id1", "id2", "id3", "id4", "id5"]
)

In [24]:
collection.get()

{'ids': ['id1', 'id2', 'id4', 'id5', 'id3'],
 'embeddings': None,
 'documents': ['This is a document about LangChain',
  'This is a reading about LlamaIndex',
  'This is a document about pandas',
  'This is another document about LangChain',
  'This is a book about Python'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'version': 0.1, 'source': 'langchain.com'},
  {'source': 'llamaindex.ai', 'version': 0.2},
  {'source': 'pandas.pydata.org', 'version': 0.4},
  {'source': 'langchain.com', 'version': 0.5},
  {'version': 0.3, 'source': 'python.org'}]}

In [25]:
collection.get(
    where={"source": "langchain.com"}
)

{'ids': ['id1', 'id5'],
 'embeddings': None,
 'documents': ['This is a document about LangChain',
  'This is another document about LangChain'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'source': 'langchain.com', 'version': 0.1},
  {'version': 0.5, 'source': 'langchain.com'}]}

In [21]:
collection.delete(
    where={"source": "python.org"},
    ids='id3'
)

In [31]:
collection.get()

{'ids': ['id1', 'id2', 'id4', 'id5', 'id3'],
 'embeddings': None,
 'documents': ['This is a document about LangChain',
  'This is a reading about LlamaIndex',
  'This is a document about pandas',
  'This is another document about LangChain',
  'This is a book about Python'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'source': 'langchain.com', 'version': 0.1},
  {'source': 'llamaindex.ai', 'version': 0.2},
  {'source': 'pandas.pydata.org', 'version': 0.4},
  {'version': 0.5, 'source': 'langchain.com'},
  {'version': 0.3, 'source': 'python.org'}]}

###chaining filters

In [33]:
collection.get(
    where={
        "$and":[
            {"source": {"$in": ["langchain.com", "llamaindex.ai"]}},
            {"version": {"$lte": 0.3}}
        ]
    }
)

{'ids': ['id1', 'id2'],
 'embeddings': None,
 'documents': ['This is a document about LangChain',
  'This is a reading about LlamaIndex'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'source': 'langchain.com', 'version': 0.1},
  {'source': 'llamaindex.ai', 'version': 0.2}]}

##filtering based on document content


In [37]:
collection.get(
    where_document={
        "$or":[
            {"$contains":"Python"},
            {"$contains":"pandas"}
        ]
    }
)

{'ids': ['id4', 'id3'],
 'embeddings': None,
 'documents': ['This is a document about pandas',
  'This is a book about Python'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'source': 'pandas.pydata.org', 'version': 0.4},
  {'version': 0.3, 'source': 'python.org'}]}

##combining meta data filter as well as document filters

In [38]:
collection.get(
     where={"version": {"$gt": 0.1}},
    where_document={
        "$or": [
            {"$contains": "LangChain"},
            {"$contains": "Python"}
        ]
    }
)

{'ids': ['id5', 'id3'],
 'embeddings': None,
 'documents': ['This is another document about LangChain',
  'This is a book about Python'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'source': 'langchain.com', 'version': 0.5},
  {'source': 'python.org', 'version': 0.3}]}